In [3]:
import pandas as pd
import numpy as np

In [4]:
filepath = './HeroesOfPymoli_Resources_purchase_data.csv'

In [5]:
df = pd.read_csv(filepath,encoding="ISO-8859-1")

In [49]:
#preview of DataFrame
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108.0,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143.0,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92.0,Final Critic,4.88
3,3,Chamassasya86,24,Male,100.0,Blindscythe,3.27
4,4,Iskosia90,23,Male,131.0,Fury,1.44


In [7]:
#identify complete rows
df.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [8]:
#review the column types
df.dtypes

Purchase ID      int64
SN              object
Age              int64
Gender          object
Item ID          int64
Item Name       object
Price          float64
dtype: object

In [9]:
#change item id from Int64 to float64
df['Item ID'] = pd.to_numeric(df['Item ID'], downcast= 'float')
df['Item ID'].dtype

dtype('float32')

In [10]:
#To confirm a unique Item ID per Item Name
#To highlight value greater than 1

df_itemsvsitemid= df.groupby('Item Name')['Item ID'].nunique().reset_index(name= "# of Item IDs")
#df_itemsvsitemid_id= df_itemsvsitemid.subset=['# of Item IDs']
df_itemsvsitemid
#def color_red_over1(df_itemsvsitemid):
    #color= 'red' if df_itemsvsitemid > 1.0 else 'black'
   # return 'color: %s' % color
#def highlight_over1(df_itemsvsitemid):
#    return ['bacground-color: yellow' if df_itemsvsitemid > 1.0 else '']

#df_itemsvsitemid.style.\
  #  applymap(color_red_over1, subset=['# of Item IDs']).\
   # applymap(highlight_over1, subset=['# of Item IDs'])


,Item Name,# of Item IDs
0,Abyssal Shard,1
1,"Aetherius, Boon of the Blessed",1
2,Agatha,1
3,Alpha,1
4,"Alpha, Oath of Zeal",1
5,"Alpha, Reach of Ending Hope",1
6,Amnesia,1
7,Apocalyptic Battlescythe,1
8,Arcane Gem,1
9,Avenger,1


In [ ]:
#source[unqiue values per group]:https://stackoverflow.com/questions/41415017/count-unique-values-using-pandas-groupby

In [17]:
#to create a dataframe to review the item Ids for "Crucifer"
cruciferIDs_df=df.loc[df['Item Name'] == 'Crucifer']
cruciferIDs_df


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
121,121,Ilosia37,23,Male,1.0,Crucifer,3.26
251,251,Iasur80,22,Male,23.0,Crucifer,1.99
454,454,Lassimla92,25,Male,1.0,Crucifer,3.26
697,697,Tyaelistidru84,19,Female,1.0,Crucifer,3.26


In [84]:
#statistic summary on item name and prices
df_statsbyItem =round(df.groupby('Item Name')['Price'].describe().reset_index(),2)
df_statsbyItem.head()

,Item Name,count,mean,std,min,25%,50%,75%,max
0,Abyssal Shard,5.0,2.67,0.0,2.67,2.67,2.67,2.67,2.67
1,"Aetherius, Boon of the Blessed",5.0,3.39,0.0,3.39,3.39,3.39,3.39,3.39
2,Agatha,6.0,3.08,0.0,3.08,3.08,3.08,3.08,3.08
3,Alpha,3.0,2.07,0.0,2.07,2.07,2.07,2.07,2.07
4,"Alpha, Oath of Zeal",3.0,4.05,0.0,4.05,4.05,4.05,4.05,4.05


In [19]:
#to review is Item ID 23.0 is for a another product
itemid23_df=df.loc[df['Item ID'] == 23]
itemid23_df

#since item 23 is only for Crucifer, it appears to be about a 40% discount value value
#Before I rename the Item Name for item Id 23.0 I will check to see if other items had discounted items but with the same id #
#if so, then we will change the id # rathe than the name

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
251,251,Iasur80,22,Male,23.0,Crucifer,1.99


In [99]:
def highlight_max(df, color='yellow'):
    #highlight max value in a column
    
    attr= 'background-color: {}'.format(color)
    if df.ndim ==1:
        is_max = df == df.max().max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = df == df.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

In [100]:
#Replaced NaN with zero because there was no std
statsbyitem_df= df_statsbyItem.fillna(0)
statsbyitem_df
#.style.apply(highlight_max, subset=["std"])
#.style.highlight_max(subset = ['std'])


,Item Name,count,mean,std,min,25%,50%,75%,max
0,Abyssal Shard,5.0,2.67,0.0,2.67,2.67,2.67,2.67,2.67
1,"Aetherius, Boon of the Blessed",5.0,3.39,0.0,3.39,3.39,3.39,3.39,3.39
2,Agatha,6.0,3.08,0.0,3.08,3.08,3.08,3.08,3.08
3,Alpha,3.0,2.07,0.0,2.07,2.07,2.07,2.07,2.07
4,"Alpha, Oath of Zeal",3.0,4.05,0.0,4.05,4.05,4.05,4.05,4.05
5,"Alpha, Reach of Ending Hope",1.0,3.58,0.0,3.58,3.58,3.58,3.58,3.58
6,Amnesia,6.0,2.18,0.0,2.18,2.18,2.18,2.18,2.18
7,Apocalyptic Battlescythe,6.0,1.97,0.0,1.97,1.97,1.97,1.97,1.97
8,Arcane Gem,3.0,3.79,0.0,3.79,3.79,3.79,3.79,3.79
9,Avenger,6.0,3.44,0.0,3.44,3.44,3.44,3.44,3.44


In [101]:
statsbyitem_df.style.apply(highlight_max, subset=['std'],axis = 0)

,Item Name,count,mean,std,min,25%,50%,75%,max
0,Abyssal Shard,5,2.67,0,2.67,2.67,2.67,2.67,2.67
1,"Aetherius, Boon of the Blessed",5,3.39,0,3.39,3.39,3.39,3.39,3.39
2,Agatha,6,3.08,0,3.08,3.08,3.08,3.08,3.08
3,Alpha,3,2.07,0,2.07,2.07,2.07,2.07,2.07
4,"Alpha, Oath of Zeal",3,4.05,0,4.05,4.05,4.05,4.05,4.05
5,"Alpha, Reach of Ending Hope",1,3.58,0,3.58,3.58,3.58,3.58,3.58
6,Amnesia,6,2.18,0,2.18,2.18,2.18,2.18,2.18
7,Apocalyptic Battlescythe,6,1.97,0,1.97,1.97,1.97,1.97,1.97
8,Arcane Gem,3,3.79,0,3.79,3.79,3.79,3.79,3.79
9,Avenger,6,3.44,0,3.44,3.44,3.44,3.44,3.44


In [102]:
# We will rename item ID 23 since this discount seems unnormal.
#It could be an error in the sytem or a new disc code.

df.loc[df['Item ID'] == 23 & df['Item Name'] == ('Crucifer'), 'Item Name'] = 'Disc.Crucifer' 
df_rev1

TypeError: cannot compare a dtyped [object] array with a scalar of type [bool]

In [63]:
cruciferIDs_df_rev1=df_rev1.loc[df['Item Name'] == 'Crucifer']
cruciferIDs_df_rev1

AttributeError: 'str' object has no attribute 'loc'

In [ ]:
#source:https://stackoverflow.com/questions/52224142/pandas-conditionally-replace-values-based-on-other-columns-values

In [50]:
#to count the number of Items ID purchased per SN
df_numofitemspur= df.groupby('SN')['Item ID'].nunique().sort_values(ascending = False).reset_index(name= "Total Purchased Products")
df_numofitemspur

,SN,Total Purchased Products
0,Lisosia93,5
1,Iral74,4
2,Idastidru52,4
3,Aina42,3
4,Iri67,3
5,Chamjask73,3
6,Iskadarya95,3
7,Phyali88,3
8,Chanastnya43,3
9,Silaera56,3


In [ ]:
#source[Name a df column]: https://stackoverflow.com/questions/29836477/pandas-create-new-column-with-count-from-groupby

In [59]:
df_numofitemsbySN= df_numofitemspur.groupby('Total Purchased Products').count().sort_values(by="SN", ascending = False)
df_numofitemsbySN

,SN
Total Purchased Products,
1,414
2,124
3,35
4,2
5,1


In [61]:
#statistic summary on Gender and Age
df_statsbyGenderAge= round(df.groupby('Gender')['Age'].describe().reset_index(),0)
df_statsbyGenderAge

,Gender,count,mean,std,min,25%,50%,75%,max
0,Female,113.0,21.0,6.0,7.0,18.0,21.0,24.0,40.0
1,Male,652.0,23.0,7.0,7.0,20.0,22.0,25.0,45.0
2,Other / Non-Disclosed,15.0,24.0,8.0,15.0,18.0,21.0,30.0,38.0


In [34]:
#source[summarystatisticsbygroup]:https://datascience.stackexchange.com/questions/22266/summary-statistics-by-category-using-python

In [ ]:
#statistic summary on Gender and Age